기상청 api와 naver 과거날씨를 받아서 미래 날씨를 예측해보자

In [5]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
scaler_data = pd.read_csv('./weather_kor4.csv')
naver_data = pd.read_csv('./new_data.csv')

In [4]:
scaler_data.head(1)

,Unnamed: 0,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1


그 외의 nan => fillna(0)

In [ ]:
# 기상청 api의 nan값 존재시 근처의 +-3 시간대가 존재한다면 선형회귀로로
# ex) 10 20 30 nan 50 60 70
data = np.array(predict_array).reshape(-1, 1)
model = LinearRegression()
x_train = np.where(~np.isnan(data))[0].reshape(-1, 1) #행렬로 변환
y_train = data[~np.isnan(data)].reshape(-1, 1)
model.fit(x_train, y_train)
X_pred = np.where(np.isnan(data))[0].reshape(-1, 1)
y_pred = model.predict(X_pred)
joblib.dump(model, 'linear_regression_model.joblib')
print("예측 결과:", y_pred.flatten()) #2차원 배열 1차원으로 

나머지 nan 값은 5개와 68개 식으로 전체 개수에 비해 미미했으므로 drop

In [ ]:
wd_direction = {"20": "NNE",
"50": "NE",
"70": "ENE",
"90": "E",
"110": "ESE",
"140": "SE",
"160": "SSE",
"180": "S",
"200": "SSW",
"230": "SW",
"250": "WSW" ,
"270": "W",
"290": "WNW",
"320": "NW",
"340": "NNW",
"360": "N"}

In [ ]:
기상청 데이터를 2018년부터 받아왔으나 naver 과거날씨를 받아오는 과정에서 naver는 과거날씨를 하루전체를 통일되게 보여주므로
2020년 기록부터 있었으므로 2019.12.29~ 2024.05.23 11:00 까지 데이터 join
24개의 시간대가 하루이므로 => 일단은 전체를 wetrTxt를 넣어놓은 다음

In [6]:
naver_data.head(1)

,aplYmd,lareaNm,mareaNm,wetrCd,wetrTxt
0,20191229.0,서울특별시,종로구,9.0,흐림/비


In [7]:
scaler_data.head(1)

,Unnamed: 0,날짜,기온,강수량,풍속_num,풍향_num,습도,현지기압,전운량,aplYmd,...,현상번호_split,weather_1,weather_2,풍속,풍향,일조량,시간당 강수량,timestamp,W1,W2
0,0,2019-12-29 00:00,1.4,0.0,1.8,50.0,57,1018.2,3.0,20191229,...,"['92', '01']",맑음,둘다아님,약한 강한 바람,NE,-1.0,없음,2019-12-29 00:00:00,1,1


In [ ]:
과거의 날씨는 그 날 하루의 평균 기상 상태가 나오므로 기상청에서 받아온 데이터와 join 시킬때
아래의 문서를 보면서 기압과 강수량, 기온을 보고 시간대에 많는 weather_1과 weather_2만듬
그렇게 해서 각 시간대별로 맑음,흐림,구름많음 비,눈,둘다안옴 판단

In [ ]:
for i in range(len(df)):
    if "05" in df.loc[i, "new_list"]: #그 시간대에 일별데이터에 눈이 포함되어있고
        if df.loc[i,"rn"] > 0 :
            df.loc[i,"weather_2"] = "눈"
        

기상청 api의 문서를 보고 
https://www.weather.go.kr/w/community/knowledge/fct-term-exp.do
맑음, 구름많음, 흐림 구분
풍향 구분
풍속 구분

In [ ]:
# 저 위의 weather_1과 weather_2 기준으로 라벨인코딩
df.reset_index(drop=True,inplace=True)
# weather_1에 라벨인코딩
encoder = LabelEncoder()
encoder.fit(df["weather_1"])
encoded_data = encoder.transform(df["weather_1"])
# # weather_2에 라벨인코딩
encoder1 = LabelEncoder()
encoder1.fit(df["weather_2"])
encoded_data1 = encoder1.transform(df["weather_2"])
# data_Frame으로 변환
encoded_data_df = pd.DataFrame(encoded_data,columns=['W1'])
# data_Frame으로 변환
encoded_data_df1 = pd.DataFrame(encoded_data1,columns=['W2'])

In [ ]:
input_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']
output_features = ['기온', '강수량', '풍속_num', '풍향_num','습도','현지기압','전운량','W1','W2']

In [ ]:
scalers = {}
for feature in input_features:
    scaler = MinMaxScaler()
    df[feature] = scaler.fit_transform(df[[feature]])
    scalers[feature] = scaler
# 내가 보고싶은 시간대 마지막 데이터가 2023-05-23 11:00 시이므로 1시간간격으로
FUTURE_TARGET = 48 

In [ ]:
# 시계열 데이터이므로 sequnece_length를 720으로 잡음
for i in tqdm(range(len(df) - sequence_length)):
    # 각 변수에 해당하는 데이터를 스케일링하여 저장
    scaled_features = []
    for feature in input_features:
        # 그 feature에 관한 값을 시퀀스길이만큼 스케일러로 변환
        scaled_feature = scalers[feature].transform(df[[feature]].iloc[i:i+sequence_length])
        scaled_features.append(scaled_feature)
    # 각 변수를 열로 가지는 2차원 배열로 변환하여 X에 추가
    X1.append(np.concatenate(scaled_features, axis=1))
    # 출력 변수는 스케일링할 필요가 없으므로 그대로 추가
    # i+sequence_length를 해서 720시간 뒤의 y값을 저장
    Y1.append(df[output_features].iloc[i+sequence_length])
X2 = np.array(X1)
Y2 = np.array(Y1)


In [ ]:
model_2_1_2_2 에 shape있음

X2와 Y2를 먼저 8대 2로 TRAIN,TEST로 나누고 TRAIN에서 또 8:2로 TRAIN,VALID 나눔

In [ ]:
# LSTM 모델 구성
# 100,50은 뉴런의 수 조절해야하는 파라미터 과적합 조심!
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(100, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(50,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(len(output_features))
])

In [ ]:
model.compile(loss='mse', optimizer='adam')

In [ ]:
# 모델 성능이 개선되지 않을때 대비
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  
    patience=20,        
    restore_best_weights=True  
)

In [ ]:
# 모델 학습
history = model.fit(
    x_train, y_train,
    epochs=5,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[early_stopping_callback]  
)

In [ ]:
y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)
# 정확도 계산
accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred_class)
print("Accuracy:", accuracy)
# 0.48

gradio(2) 파일 2번째 셀
gradio 실행시켜서 사이트열기

In [ ]:
모델 입출력변수 통일? 각변수 각각 스케일러? epoch 5? 더 복잡한 모델 층? , 입력변수들의 상관관계 가중치를 넣으면 더 좋지않았을까